In [2]:
import numpy as np
import wandb
from tqdm import tqdm
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

In [10]:
# Load data from .tsv files
train_df = pd.read_csv('cdr_train_data', delimiter='\t')
test_df = pd.read_csv('test_data', delimiter='\t')

# Download the stopwords list
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# Apply the remove_stopwords function to the 'sentence' column
train_df['sentence'] = train_df['sentence'].apply(remove_stopwords)
test_df['sentence'] = test_df['sentence'].apply(remove_stopwords)

X_train = train_df['sentence'].tolist()
y_train = train_df['label'].tolist()

X_test = test_df['sentence'].tolist()
y_test = test_df['label'].tolist()

# Get BioBERT embeddings for the data
# try:
#     X_train_embeddings = np.vstack([get_specific_token_embeddings(sentence) for sentence in tqdm(X_train)])
#     X_test_embeddings = np.vstack([get_specific_token_embeddings(sentence) for sentence in tqdm(X_test)])
#     if(len(X_train_embeddings)==0):
#         print(sentence)
# except Exception as e:
#     print(e)
#     pass
    

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import pandas as pd

# Reading the training embeddings from the text file
X_train_embeddings = pd.read_csv('X_train_embeddings_CDR.txt', sep='\t', header=None)

# Reading the test embeddings from the text file
X_test_embeddings = pd.read_csv('X_test_embeddings.txt', sep='\t', header=None)


# SVM

In [ ]:
sweep_config = {
    'name': 'svm-hyperparameter-sweep',
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'f1_weighted',
        'goal': 'maximize'   
    },
    'parameters': {
        'kernel': {
            'values': ['rbf', 'linear', 'poly']
        },
        'C': {
            'values': [0.1, 1, 10]
        },
        'degree': {
            'values': [4, 30, 53]
        },
        'gamma': {
            'values': ['scale', 'auto']
        }
    }
}


In [ ]:
def train():
    # Initialize a new run
    run = wandb.init()

    # Get hyperparameters from this run's configuration
    config = wandb.config
    
    # Train SVM
    clf = SVC(kernel=config.kernel, C=config.C, degree=config.degree, 
              gamma=config.gamma, class_weight={0:1, 1:30}, probability=True)
    clf.fit(X_train_embeddings, y_train)

    # Evaluate
    y_pred = clf.predict(X_test_embeddings)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Log metrics to wandb
    wandb.log({"classification_report": report, "f1_avg": report['macro avg']['f1-score']})

    # Finish the run
    wandb.finish()


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="svm_hyperparameter_tuning_latest")
wandb.agent(sweep_id, function=train)

# XGBoost

In [12]:
sweep_config = {
    'name': 'xgboost-hyperparameter',
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'f1_weighted',
        'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.1, 0.01, 0.2,]
        },
        'max_depth': {
            'values': [5, 10, 20, 40]
        },
        'n_estimators': {
            'values': [100, 200, 400]
        },
        'scale_pos_weight': {
            'values': [100, 200, 500]
        },
        'gamma': {
            'values': [0.1, 0.2, 0.3]
        }
    }
}


In [13]:
def train():
    # Initialize a new run
    run = wandb.init()

    # Get hyperparameters from this run's configuration
    config = wandb.config
    
    # Train XG BOOST
    clf = xgb.XGBClassifier(scale_pos_weight=config.scale_pos_weight, max_depth=config.max_depth, learning_rate=config.learning_rate, n_estimators=config.n_estimators, gamma=config.gamma)

    clf.fit(X_train_embeddings, y_train)
    

    # Evaluate
    y_pred = clf.predict(X_test_embeddings)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Log metrics to wandb
    wandb.log({"classification_report": report, "f1_avg": report['macro avg']['f1-score']})

    # Finish the run
    wandb.finish()


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="xgboost_hyperparameter_tuning_cdr_train")
wandb.agent(sweep_id, function=train)

Create sweep with ID: rkvy4jc9
Sweep URL: https://wandb.ai/nebha/xgboost_hyperparameter_tuning_cdr_train/sweeps/rkvy4jc9


wandb: Agent Starting Run: p8f2ue40 with config:
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 10
wandb: 	n_estimators: 200
wandb: 	scale_pos_weight: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

f1_avg,▁
f1_avg,0.51639


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 83gsnfsm with config:
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 40
wandb: 	n_estimators: 400
wandb: 	scale_pos_weight: 500
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: obwahbmu with config:
wandb: 	gamma: 0.2
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 20
wandb: 	n_estimators: 100
wandb: 	scale_pos_weight: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

f1_avg,▁
f1_avg,0.46591


wandb: Agent Starting Run: lbf1bk9v with config:
wandb: 	gamma: 0.2
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 40
wandb: 	n_estimators: 200
wandb: 	scale_pos_weight: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/ubuntu/miniconda3/envs/hugging/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

# Random Forest

In [4]:
sweep_config = {
    'name': 'random_forest-hyperparameter',
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'f1_weighted',
        'goal': 'maximize'   
    },
    'parameters': {
        'n_jobs': {
            'values': [2, 5, 10]
        },
        'max_depth': {
            'values': [10, 50 , 100]
        },
        'n_estimators': {
            'values': [100, 200, 400]
        },
        'max_leaf_nodes': {
            'values': [100, 200, 500]
        },
        'max_features': {
            'values': ['auto', 'sqrt']
        }
    }
}


In [5]:
def train():
    # Initialize a new run
    run = wandb.init()

    # Get hyperparameters from this run's configuration
    config = wandb.config
    
    # Train XG BOOST
    clf = RandomForestClassifier(n_estimators=config.n_estimators, max_depth=config.max_depth, max_leaf_nodes= config.max_leaf_nodes, n_jobs= config.n_jobs, max_features=config.max_features, class_weight={0:1, 1:30}, verbose=True)

    clf.fit(X_train_embeddings, y_train)
    

    # Evaluate
    y_pred = clf.predict(X_test_embeddings)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Log metrics to wandb
    wandb.log({"classification_report": report, "f1_avg": report['macro avg']['f1-score']})

    # Finish the run
    wandb.finish()


In [6]:
sweep_id = wandb.sweep(sweep=sweep_config, project="random_forest_hyperparameter_tuning")
wandb.agent(sweep_id, function=train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vhhlkewx
Sweep URL: https://wandb.ai/nebha/random_forest_hyperparameter_tuning/sweeps/vhhlkewx


wandb: Agent Starting Run: 26khvjsv with config:
wandb: 	max_depth: 50
wandb: 	max_features: sqrt
wandb: 	max_leaf_nodes: 100
wandb: 	n_estimators: 200
wandb: 	n_jobs: 10
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs21d002 (nebha). Use `wandb login --relogin` to force relogin


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.6s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    2.9s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    3.3s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.1s finished


f1_avg,▁
f1_avg,0.58169


...

f1_avg,▁
f1_avg,0.59527


wandb: Agent Starting Run: e9x6x5kg with config:
wandb: 	max_depth: 50
wandb: 	max_features: auto
wandb: 	max_leaf_nodes: 500
wandb: 	n_estimators: 200
wandb: 	n_jobs: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run e9x6x5kg errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: ERROR Run e9x6x5kg errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: Agent Starting Run: mzavaf6u with config:
wandb: 	max_depth: 100
wandb: 	max_features: auto
wandb: 	max_leaf_nodes: 100
wandb: 	n_estimators: 400
wandb: 	n_jobs: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run mzavaf6u errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: ERROR Run mzavaf6u errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: Agent Starting Run: th995xli with config:
wandb: 	max_depth: 50
wandb: 	max_features: auto
wandb: 	max_leaf_nodes: 200
wandb: 	n_estimators: 100
wandb: 	n_jobs: 10
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run th995xli errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: ERROR Run th995xli errored: InvalidParameterError("The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.")
wandb: Sweep Agent: Waiting for job.
wandb: Ctrl + C detected. Stopping sweep.


# Cat Boost

In [10]:
sweep_config = {
    'name': 'catboost-hyperparameter',
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'f1_weighted',
        'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.1, 0.01, 0.2,]
        },
        'depth': {
            'values': [5, 10, 20, 40]
        },
        'iterations': {
            'values': [100, 200, 500]
        }
    }
}


In [11]:
def train():
    # Initialize a new run
    run = wandb.init()

    # Get hyperparameters from this run's configuration
    config = wandb.config
    
    # Train XG BOOST
    clf = CatBoostClassifier(iterations=config.iterations, 
                         depth=config.depth, 
                         learning_rate=config.learning_rate, 
                         verbose=200,
                         class_weights=[1, 30])
    clf.fit(X_train_embeddings, y_train)
    

    # Evaluate
    y_pred = clf.predict(X_test_embeddings)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Log metrics to wandb
    wandb.log({"classification_report": report, "f1_avg": report['macro avg']['f1-score']})

    # Finish the run
    wandb.finish()


In [12]:
sweep_id = wandb.sweep(sweep=sweep_config, project="cat_boost_hyperparameter_tuning")
wandb.agent(sweep_id, function=train)

Create sweep with ID: rcbxhhbx
Sweep URL: https://wandb.ai/nebha/cat_boost_hyperparameter_tuning/sweeps/rcbxhhbx


wandb: Agent Starting Run: oe6le8h5 with config:
wandb: 	depth: 5
wandb: 	iterations: 500
wandb: 	learning_rate: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


0:	learn: 0.6321714	total: 77.7ms	remaining: 38.8s
200:	learn: 0.0321011	total: 5.45s	remaining: 8.1s
400:	learn: 0.0104495	total: 10.8s	remaining: 2.66s
499:	learn: 0.0082219	total: 13.4s	remaining: 0us


f1_avg,▁
f1_avg,0.6686


wandb: Agent Starting Run: ky80384b with config:
wandb: 	depth: 20
wandb: 	iterations: 500
wandb: 	learning_rate: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

f1_avg,▁
f1_avg,0.67621


wandb: Agent Starting Run: p387gyzc with config:
wandb: 	depth: 20
wandb: 	iterations: 200
wandb: 	learning_rate: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run p387gyzc errored: CatBoostError('/src/catboost/catboost/private/libs/options/oblivious_tree_options.cpp:128: Maximum tree depth is 16')
wandb: ERROR Run p387gyzc errored: CatBoostError('/src/catboost/catboost/private/libs/options/oblivious_tree_options.cpp:128: Maximum tree depth is 16')
wandb: Agent Starting Run: xdogdzjd with config:
wandb: 	depth: 10
wandb: 	iterations: 500
wandb: 	learning_rate: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


0:	learn: 0.6011231	total: 588ms	remaining: 4m 53s


wandb: Ctrl + C detected. Stopping sweep.
